In [1]:
import os, shutil, itertools, yaml, kornia, torchvision, sys, copy, math
from functools import partial
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import seaborn as sns
from PIL import Image
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr
from time import time

In [2]:
from inrnet import args as args_module
from inrnet import inn, experiments, optim, util, losses, models
from inrnet.data import dataloader
from inrnet.experiments import depth
import inrnet.inn.functional as inrF
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

## interactive

In [ ]:
torch.load("/data/vision/polina/users/clintonw/code/diffcoord/results/inet/results.txt")

In [ ]:
CUDA_VISIBLE_DEVICES=3 python train.py -j=inet2 -c=inet

In [ ]:
nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))(torch.randn(1,3,8,8)).shape

In [ ]:
nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2))(torch.randn(1,3,8,8)).shape

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
base = torchvision.models.segmentation.fcn_resnet50(pretrained=True)

In [97]:
base = torch.load(osp.expanduser('~/code/diffcoord/temp/upernet_convnext.pth'))

In [208]:
dataset = torchvision.datasets.Cityscapes(DS_DIR+'/cityscapes',
            split='val', mode='coarse', target_type='semantic',
            transform=transforms.ToTensor())

In [209]:
for out in dataset:
    break

In [203]:
img = plt.imread('/data/vision/polina/scratch/clintonw/datasets/cityscapes/gtCoarse/val/frankfurt/frankfurt_000001_080091_gtCoarse_labelIds.png')

In [ ]:
base = torchvision.models.efficientnet_b0(pretrained=True)
base

In [42]:
DS_DIR = "/data/vision/polina/scratch/clintonw/datasets"
ds = torchvision.datasets.CIFAR10(root=DS_DIR, transform=transforms.ToTensor())

In [78]:
torch.cuda.empty_cache()
coords = util.meshgrid_coords(64,64)
#spacing = torch.tensor((4/127,4/127), device="cuda")
values = torch.randn(coords.size(0),6, device=coords.device)

In [ ]:
ax = sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.5,.8,10)+torch.randn(10)/8, label="qmc");
sns.lineplot(x=torch.linspace(128,256,10), y=torch.linspace(.1,.8,10)+torch.randn(10)/8, label="grid", ax=ax);
ax.set_xlabel("Number of sample points");
ax.set_ylabel("ImageNet top-1 error");

In [246]:
dataset = torchvision.datasets.Cityscapes(DS_DIR+'/cityscapes', split='val', mode='coarse', target_type='semantic')

In [ ]:
2975, 500

In [247]:
len(dataset)

500

In [ ]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/diffcoord/inrnet")
query_string = ' INR'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)

In [257]:
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
ds = torchvision.datasets.Cityscapes(DS_DIR+'/cityscapes',
    split='val', mode='coarse', target_type='semantic',
    transform=trans)


In [258]:
for img,seg in ds:
    break

In [260]:
img.shape

torch.Size([3, 1024, 2048])

## inet12

In [ ]:
from robustness.tools.imagenet_helpers import ImageNetHierarchy, common_superclass_wnid
in_path = '/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch'
in_hier = ImageNetHierarchy(in_path, in_path)
superclass_wnid = common_superclass_wnid('big_12')
class_ranges, label_map = in_hier.get_subclasses(superclass_wnid,
                                                 balanced=True)

sub_to_super = {}
for supercls,subclasses in enumerate(class_ranges):
    for subcls in subclasses:
        sub_to_super[subcls] = supercls

big12path = f"{DS_DIR}/inrnet/big_12.pkl"
pickle.dump((label_map, class_ranges, sub_to_super), open(big12path, 'wb'))

In [ ]:
pickle.load(open(big12path, 'rb'))

In [146]:
label_to_super = {}
for supercls,subclasses in enumerate(class_labels):
    for subcls in subclasses:
        label_to_super[subcls] = supercls

In [165]:
big12path = f"{DS_DIR}/inrnet/big_12_labels.pkl"
pickle.dump((split_paths_by_cls, class_labels, label_to_super), open(big12path, 'wb'))

In [149]:
label = osp.basename(osp.dirname(subpaths[0]))

In [151]:
subpaths_by_cls = [[] for _ in range(12)]
for path in subpaths:
    label = osp.basename(osp.dirname(path))
    if label in label_to_super.keys():
        subpaths_by_cls[label_to_super[label]].append(path)

In [162]:
split_paths_by_cls = {'train':[p[:800] for p in subpaths_by_cls],
                     'test':[p[800:] for p in subpaths_by_cls]}

In [159]:
for p in subpaths_by_cls:
    np.random.shuffle(p)

In [111]:
subpaths = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/val.txt",
            "r").read().split('\n')

In [136]:
labels = open("/data/vision/polina/scratch/clintonw/datasets/imagenet_pytorch/labels.txt",
            "r").read().split('\n')

In [137]:
labels = [l[:l.find(',')] for l in labels[:-1]]

In [138]:
labels[953]

'n07753275'

In [ ]:
class_labels = [np.array(labels)[list(cr)].tolist() for cr in class_ranges]

## Figs

In [ ]:
xy = util.generate_quasirandom_sequence(d=2, n=256)
plt.rcParams["figure.figsize"] = (2,1)
fig,ax = plt.subplots()
ax.scatter(xy[:,0], xy[:,1], s=2, c='k')
ax.plot();

In [ ]:
kernel = model.features[0][0].weight[3,1].detach()
plt.rcParams["figure.figsize"] = (.5,.5)
fig,ax = plt.subplots()
print(kernel.min().item(), kernel.max().item())
ax.imshow(kernel, vmin=-2.9, vmax=2.9);
plt.axis('off');

In [100]:
from scipy.interpolate import RectBivariateSpline as Spline2D
K = [2.5,2.5]
h,w=3,3
bbox = (-K[0]/2, K[0]/2, -K[1]/2, K[1]/2)
x,y = (np.linspace(bbox[0]/h*(h-1), bbox[1]/h*(h-1), h),
       np.linspace(bbox[2]/w*(w-1), bbox[3]/w*(w-1), w))
bs = Spline2D(x,y, kernel, bbox=bbox, kx=2,ky=2, s=0)
tx,ty,c = [torch.tensor(z).float() for z in bs.tck]
c = c.reshape(h,w)

In [106]:
H = 50
xy = util.meshgrid_coords(H,H).cpu()
w_oi = []
X = xy[:,0].unsqueeze(1)
Y = xy[:,1].unsqueeze(1)
px = py = 2

values, kx = (tx<=X).min(dim=-1)
values, ky = (ty<=Y).min(dim=-1)
kx -= 1
ky -= 1
kx[values] = tx.size(-1)-px-2
ky[values] = ty.size(-1)-py-2

Ctrl = c.view(1, *c.shape[-2:])
for z in range(X.size(0)):
    D = Ctrl[:, kx[z]-px : kx[z]+1, ky[z]-py : ky[z]+1].clone()

    for r in range(1, px + 1):
        try:
            alphax = (X[z,0] - tx[kx[z]-px+1:kx[z]+1]) / (
                tx[2+kx[z]-r:2+kx[z]-r+px] - tx[kx[z]-px+1:kx[z]+1])
        except RuntimeError:
            print("input off the grid")
        for j in range(px, r - 1, -1):
            D[:,j] = (1-alphax[j-1]) * D[:,j-1] + alphax[j-1] * D[:,j].clone()

    for r in range(1, py + 1):
        alphay = (Y[z,0] - ty[ky[z]-py+1:ky[z]+1]) / (
            ty[2+ky[z]-r:2+ky[z]-r+py] - ty[ky[z]-py+1:ky[z]+1])
        for j in range(py, r-1, -1):
            D[:,px,j] = (1-alphay[j-1]) * D[:,px,j-1].clone() + alphay[j-1] * D[:,px,j].clone()

    w_oi.append(D[:,px,py])

w = torch.stack(w_oi).view(xy.size(0))

In [107]:
k = w.reshape(H,H)

In [ ]:
fig,ax = plt.subplots()
print(k.min().item(), k.max().item())
ax.imshow(k, vmin=-2.9, vmax=2.9);
plt.axis('off');

## slurm

In [45]:
coco_len = 118287
kitti_len = 80896
horse_len = 1067
zebra_len = 1334
inet_len = 50000
in_tr_len = 50000

In [ ]:
configs/convnext/upernet_convnext_tiny
configs/_base_/models/upernet_convnext.py

In [ ]:
for ix in range(0,places_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/places/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"place{ix//800}", "fit_places", ix)

In [ ]:
for ix in range(0,in_tr_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_train/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_in_tr", ix)

In [ ]:
for ix in range(0,3475,100):
    if i >= 2975:
        i-=2975
        s = 'test'
    else:
        s='train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cityscapes/{s}_{i+1}.pt"):
        print("sh fit_inr.sh", f"city{ix//100}", "fit_city", ix)

In [ ]:
for ix in range(0,12000,100):
    c,i=ix//1000, ix%1000
    if i >= 800:
        i-=800
        s = 'test'
    else:
        s='train'
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/inet12/{c}/{s}_{i+20}.pt"):
        print("sh fit_inr.sh", f"inet{ix//100}", "fit_i12", ix)

In [ ]:
for ix in range(0,inet_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/imagenet1k_val/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"inet{ix//800}", "fit_inet", ix)

In [ ]:
for ix in range(0,coco_len,1600):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/coco/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"coco{ix//1600}", "fit_coco", ix)

In [ ]:
for ix in range(0,kitti_len,800):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_{ix+63}.pt"):
        print("sh fit_inr.sh", f"skit{ix//800}", "fit_kitti", ix)

In [ ]:
for ix in range(0,zebra_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/zebra/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"zebra{ix//128}", "fit_zebra", ix)

In [ ]:
for ix in range(0,horse_len,128):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/horse/siren_{ix+15}.pt"):
        print("sh fit_inr.sh", f"horse{ix//128}", "fit_horse", ix)

In [ ]:
for ix in range(0,10000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_test/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cit{ix//5000}", "fit_ciftest", ix)
for ix in range(0,50000,5000):
    if not osp.exists(f"/data/vision/polina/scratch/clintonw/datasets/inrnet/cifar10_train/siren_{ix+999}.pt"):
        print("sh fit_inr.sh", f"cif{ix//5000}", "fit_cifar", ix)

In [ ]:
img = plt.imread("/data/vision/polina/scratch/clintonw/datasets/inrnet/horse2zebra/testA/n02381460_9260.jpg")

In [ ]:
"/data/vision/polina/scratch/clintonw/datasets/coco/annotations/panoptic_train2017"

In [ ]:
osp.exists(f"/data/vision/polina/users/clintonw/code/diffcoord/temp/kitti/siren_{ix+63}.pt")

In [60]:
paths = sorted(util.glob2("/data/vision/polina/scratch/clintonw/datasets/inrnet/kitti/siren_*.pt"))